# Pre-processing & RF & SVM

This notebook objective is to train, test and evaluate the performance of a Random Forest (RF) and a Support Vector Machine (SVM) model for predicting future `alert_11` values.

### Data loading

In [1]:
import pandas as pd

df = pd.read_parquet('./../data/dataset.parquet', engine='pyarrow')

In [2]:
# Check the dataset has been imported correctly
df.shape

(679045, 17)

## Pre-processing steps

### Dataset optimisation

The dataset has previously been utilised in a predictive maintenance study (*docs/introduction_paper*). In fact, it was retrieved directly from the GitHub repository associated with that study. To fully understand the dataset, it is essential to refer to the corresponding research paper, which indicates that the data has already undergone pre-processing. The paper states:  

> Depending on the changes detected by sensors, the irregular sampling time is addressed by resampling the time series with a frequency of 5 seconds, which gives a good trade-off between memory occupation and signal resolution. Missing values are filled in using the last available observation because the absence of new measurements indicates no change in the recorded value.  

Additionally, it highlights the original presence of more than one **alert feature**. However, due to the following reasons, only `alert_11` is kept in the dataset:  

> Figure 1 presents the distribution of the alert codes most relevant according to the manufacturer. Most of them are rare (i.e., are observed less than 10 times), and alerts 11 (platform motor inverter protection) and 34 (machine in emergency condition) are the most common ones. Alert 34 is thrown when the operator presses an emergency button. However, this occurrence heavily depends on human behavior because the operators often use the emergency button as a quick way to turn the machine off, as confirmed by the plant manager. For this reason, alert 34 is discarded, and the prediction task focuses only on alert 11.  

Furthermore, an important insight from the paper is that `session_counter` and `time_to_failure` were not part of the original machine-generated dataset. Instead, they are engineered features introduced by the researchers, with their implementations outlined in the paper through pseudo-code (`session_counter`) and a brief description (`time_to_failure`):  

> Intuitively, a session start time is introduced when at least one motion variable increases its value, and a session end time is created when all motion variables have decreased their value in the last ten minutes. Algorithm 1 specifies how sessions are defined.

> After pre-processing, it is possible to compute the distance of each data sample within a work session to the successive alert, i.e., the Time To Failure
(TTF).

Therefore, based on these pieces of information, it is evident that both `time_to_failure` and `session_counter` are not relevant to the classification task, since they were generated by the researchers. Consequently, these features are removed in this section to ensure a closer real-world scenario analysis.

In [3]:
df.drop(columns=['session_counter', 'time_to_failure'], inplace=True)

In [4]:
# Display the dataset to ensure the columns have been removed
df.head()

,Timestamp,Flag roping,Platform Position [°],Platform Motor frequency [HZ],Temperature platform drive [°C],Temperature slave drive [°C],Temperature hoist drive [°C],Tensione totale film [%],Current speed cart [%],Platform motor speed [%],Lifting motor speed [RPM],Platform rotation speed [RPM],Slave rotation speed [M/MIN],Lifting speed rotation [M/MIN],alert_11
0,2021-06-07 04:14:30.742,31.0,115.0,5200.0,18.0,22.0,18.0,181.0,0.0,100.0,0.0,84.0,116.0,0.0,0.0
1,2021-06-07 04:14:35.742,31.0,115.0,5200.0,18.0,22.0,18.0,181.0,0.0,100.0,0.0,84.0,116.0,0.0,0.0
2,2021-06-07 04:14:40.742,31.0,115.0,5200.0,18.0,22.0,18.0,181.0,0.0,100.0,0.0,84.0,116.0,0.0,0.0
3,2021-06-07 04:14:45.742,31.0,115.0,5200.0,18.0,22.0,18.0,181.0,0.0,100.0,0.0,84.0,116.0,0.0,0.0
4,2021-06-07 04:14:50.742,31.0,115.0,5200.0,18.0,22.0,18.0,181.0,0.0,100.0,0.0,84.0,116.0,0.0,0.0


As is common in most time-series datasets, the `Timestamp` column must be set as the index of the dataset. This allows for efficient time-based operations, such as resampling, sliding window calculations, and trend analysis, while preserving the chronological structure of the data.

In [5]:
df.set_index('Timestamp', inplace=True)

In [6]:
# Check if Timestamp has become the index of the dataset
df.index.name

'Timestamp'

### Features extraction

In this section, the label `y` and the features `X` for the models are defined, with `alert_11` serving as the target variable (`y`) and all other columns being designated as features (`X`). This decision is based on the fact that `alert_11` represents the primary event of interest, aligning with the original study’s objective. Moreover, by including all other columns as features, the models can leverage the full range of available data to identify patterns and relationships that may contribute in predicting `alert_11`.

In [7]:
# State the label and the features
import numpy as np

label = np.array(['alert_11'])
features = np.array(df.columns.difference(label))

print(f"-> Label (shape={label.shape}): {label}")
print(f"-> Features (shape={features.shape}): {features}")

-> Label (shape=(1,)): ['alert_11']
-> Features (shape=(13,)): ['Current speed cart [%]' 'Flag roping' 'Lifting motor speed [RPM]'
 'Lifting speed rotation [M/MIN]' 'Platform Motor frequency [HZ]'
 'Platform Position [°]' 'Platform motor speed [%]'
 'Platform rotation speed [RPM]' 'Slave rotation speed [M/MIN]'
 'Temperature hoist drive [°C]' 'Temperature platform drive [°C]'
 'Temperature slave drive [°C]' 'Tensione totale film [%]']


In [8]:
# Extract and assign the label and the features, X and y
X = df[features]
y = df[label]

print(f"-> X (shape={X.shape})")
print(f"-> y (shape={y.shape})")

-> X (shape=(679045, 13))
-> y (shape=(679045, 1))


### Sliding window

Creating sliding windows as a pre-processing step is essential for capturing temporal dependencies in the dataset, allowing the models to analyse sequences of past sensor readings rather than isolated time points. 
Since machine failures and alerts often develop gradually, a single timestamp may not provide enough context for accurate predictions. However, by structuring the data into overlapping time windows, the model can learn meaningful trends and relationships that contribute to the occurrence of `alert_11`. 
This approach mimics real-world decision-making, where operators and automated systems consider historical data before identifying potential issues. 
Additionally, sliding windows help handle lag effects, ensuring that early warning signs (such as changes in temperature, speed, or vibrations) are accounted for. Furthermore, this method prevents data leakage by ensuring that predictions rely only on past information, making the model more robust for real-world deployment.

In [9]:
# Prepare the label and features for the window
X = X.to_numpy()
y = y.to_numpy().flatten()

In [10]:
# Import the numpy library for numerical operations, particularly with arrays
import numpy as np

def window(X_data, y_data, width: int, shift: int):
    
    # Initialise empty lists to store the generated windows
    X_wins, y_wins = [], []

    # Loop through each index and corresponding (X, y) pair in the data
    for index, (X, y) in enumerate(zip(X_data, y_data)):

        # Check if there is enough data remaining to create a full window plus the prediction shift
        if (index + width + shift) <= X_data.shape[0]:

            # Define a slice object that captures the future range where we want to predict
            window = slice((index + width), (index + width + shift))

            # Create an input window: take 'width' number of X_data points starting from 'index'
            X_wins.append(X_data[index: index + width])

            # For labels (y), grab the values in the 'shift' future window
            y_values_shift = y_data[window]
            
            # If any of the future y values is '1', label the window as 1 otherwise, label it as 0
            y_wins.append(int(np.any(y_values_shift == 1)))

    # Convert the lists of windows into numpy arrays for efficient processing
    X_wins = np.array(X_wins)
    y_wins = np.array(y_wins)

    # Return the X windows reshaped as (number of samples, flattened window) and the corresponding y labels
    return X_wins.reshape(X_wins.shape[0], -1), y_wins.flatten()

In [11]:
# State the variables and the size of the window
X_wins, y_wins = window(X, y, width=120, shift=180)

### Random Under Sampler (RUS)

As stated in the paper:  

> Alerts are anomalies and thus, by definition, rarer than normal behaviors.  

This observation is supported by the exploratory data analysis (EDA) conducted in this study, which confirms that the dataset is highly imbalanced. Specifically, `alert_11` consists of **677,652 instances of 0s** and only **1,393 instances of 1s**, corresponding to **99.8%** and **0.2%** of the dataset, respectively.  

This class imbalance issue was also acknowledged in the paper. Given that our study shares the same objective, **Random Under-Sampling (RUS)** is considered an appropriate method for balancing the dataset. As described in the paper:  

> The algorithm (RUS) randomly selects and removes observations from the majority class until it achieves the desired equilibrium between the two classes. In the case of the wrapping machine, RUS is applied separately on each train set (comprising 4 folds) and test set (1 fold) for each combination of RW and PW sizes, to prevent the presence of similar data in the train and test sets (i.e., partially overlapping data).  

By employing RUS to the **sliding window** data, we ensure that the models are trained on a more balanced dataset, reducing the bias toward the majority class (`alert_11 = 0`) and improving its ability to correctly predict rare alert occurrences.

In [12]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

X_res, y_res = rus.fit_resample(X_wins, y_wins)
np.unique(y_res, return_counts=True)

(array([0, 1]), array([3324, 3324]))

After applying Random Under-Sampling (RUS), it is not usually expected that the number of positive cases (1s) would exceed what was originally present in the raw dataset (1,393). However, in this case, the result is correct because the windowing process alters the distribution of labels. 

Initially, the dataset contained **677,652 instances of 0** and only **1,393 instances of 1** for the `alert_11` feature. However, when the **windowing function** is applied, each window is labelled as 1 if any future value within the shift period is 1. As a result, a single occurrence of 1 in the original data can lead to multiple windows being labelled as 1. Therefore, after windowing, the number of windows labelled as 1 becomes much larger than the original 1,393. 

**Random Under-Sampling** then balances the two classes by reducing the majority class (0) down to match the number of minority instances (1) found after windowing, without creating any new 1s. This is why there is a result of 3,324 instances for both 0 and 1 after resampling.

# Modelling

To establish the credibility of this study, three models (two in this notebook, another in the *Pre-processing & LSTM*) from the original research are implemented. However, it is important to emphasise that this study is not merely a replication of the dataset-associated paper. While the objectives and methodological approaches remain the same, the **techniques** and **models** **implementations slightly differ** from the original study.  

One key distinction is that the code in this study is designed to be **simpler and more accessible**, making it easier to understand compared to the implementation in the paper’s GitHub repository ([GitHub Repository](https://github.com/nicolopinci/polimi_failure_prediction)). Despite its simplified approach, the effectiveness of this study remains uncompromised, as the results align closely with those presented in the original research.  

By incorporating both **traditional machine learning models (RF, SVM)** and **a deep learning approach (LSTM)**, this study provides a comprehensive evaluation of different techniques in predictive maintenance, ensuring a well-rounded assessment of classification performance on the given dataset.

Since this is a classification task, all models will be evaluated using **accuracy, precision, recall, and F1-score**, as these metrics provide a comprehensive assessment of a model's performance, particularly in detecting rare alert occurrences.  

Additionally, **stratified and non-stratified 5-fold cross-validation** will be used for models' validation. This sightly differs from the original work as, in this work, there is a prefernce in the use of stratified 5-fold cross-validation rather than 5-fold cross-validation.

> The validation procedure is also adapted to the characteristics of the different use cases. In the wrapping machine dataset, there are only 13 alarms, which yield ≈500 failure RWs in the whole time series. Thus, the number of failure RWs in the test set would be too small to test adequately the performances. Thus, we adopt a training and evaluation procedure based on k-fold cross-validation (with k = 5).

The reason of this choice is that stratified 5-fold cross-validation ensures that each fold maintains the original class distribution, which is especially important for imbalanced datasets. In contrast, standard 5-fold cross-validation may create folds with uneven class ratios, leading to biased model evaluation.

### Random Forest (RF)

The code presented below displays two Random Forest (RF) models developed for the same prediction task. The reason for implementing both versions is due to the differences in their results, caused by a small change in the validation approach.

The first model uses **Stratified 5-Fold Cross-Validation**, the same technique applied when training and evaluating the SVM and LSTM models. This method worked effectively for those two models, producing results closely aligned with those reported in the original paper. However, this was not the case for the RF model. The results appeared unrealistically high, with an `F1-score` reaching `0.995`, compared to `0.577` reported in the original paper. Such a discrepancy suggests that the model is likely overfitting.

To address this, a switch to **standard 5-Fold Cross-Validation** was made, following the original work intrustion, which produced more reliable results. Although the `F1-score` dropped to `0.40`, this outcome is more plausible and preferable to a near-perfect score, which can mask underlying issues. Moreover, the classification report further confirms that the model performs reasonably well overall, but struggles with the minority class, a known limitation in imbalanced datasets, despite the implementation of RUS.

#### RF with Stratified 5-fold Cross Validation

In [13]:
# Model implementation
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)

In [14]:
# Implement Stratified 5-Fold Cross Validation
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [15]:
# Lists to store metrics for each fold
accuracy_list = []
precision_list = []
recall_list = []
f1_list = []

In [16]:
# Perform stratified 5-fold cross-validation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

for train_index, val_index in kf.split(X_res, y_res):
    X_train, X_val = X_res[train_index], X_res[val_index]
    y_train, y_val = y_res[train_index], y_res[val_index]
    
    # Train the model
    rf_model.fit(X_train, y_train)
    
    # Predict on validation fold
    y_pred_fold = rf_model.predict(X_val)
    
    # Calculate and store metrics
    accuracy_list.append(accuracy_score(y_val, y_pred_fold))
    precision_list.append(precision_score(y_val, y_pred_fold))
    recall_list.append(recall_score(y_val, y_pred_fold))
    f1_list.append(f1_score(y_val, y_pred_fold))

In [17]:
# Print mean metrics across the 5 folds
import numpy as np

print("Mean of Metrics across 5 folds:")
print(f"Accuracy:  {np.mean(accuracy_list):.4f}")
print(f"Precision: {np.mean(precision_list):.4f}")
print(f"Recall:    {np.mean(recall_list):.4f}")
print(f"F1-Score:  {np.mean(f1_list):.4f}")

Mean of Metrics across 5 folds:
Accuracy:  0.9952
Precision: 0.9905
Recall:    1.0000
F1-Score:  0.9952


In [18]:
# Generate overall classification report
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

y_pred = cross_val_predict(rf_model, X_res, y_res, cv=kf)

print("Classification Report (based on cross-validated predictions):")
print(classification_report(y_res, y_pred))


Classification Report (based on cross-validated predictions):
              precision    recall  f1-score   support

           0       1.00      0.99      1.00      3324
           1       0.99      1.00      1.00      3324

    accuracy                           1.00      6648
   macro avg       1.00      1.00      1.00      6648
weighted avg       1.00      1.00      1.00      6648



#### RF with 5-fold Cross Validation

In [22]:
# Model implementation
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)

In [23]:
# Define the scoring metrics to evaluate
scoring = ['accuracy', 'precision', 'recall', 'f1']

In [24]:
# Implement 5-fold-cross-validation on the model
from sklearn.model_selection import cross_validate

cv_results = cross_validate(rf_model, X_res, y_res, cv=5, scoring=scoring, return_estimator=True)

In [25]:
# Print mean metrics across folds
print("Mean of Metrics across 5 folds:")
for metric in scoring:
    mean_score = cv_results['test_' + metric].mean()
    print(f"{metric.capitalize()}: {mean_score:.4f}")

Mean of Metrics across 5 folds:
Accuracy: 0.6466
Precision: 0.9151
Recall: 0.3023
F1: 0.4034


In [26]:
# Model implementation
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)

In [27]:
# Get cross-validated predictions to print a classification report
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

y_pred = cross_val_predict(rf_model, X_res, y_res, cv=5)

print("Classification Report (based on cross-validated predictions):")
print(classification_report(y_res, y_pred))

Classification Report (based on cross-validated predictions):
              precision    recall  f1-score   support

           0       0.59      0.99      0.74      3324
           1       0.97      0.30      0.46      3324

    accuracy                           0.65      6648
   macro avg       0.78      0.65      0.60      6648
weighted avg       0.78      0.65      0.60      6648



### Support Vector Machine (SVM)

In [33]:
# Perform stratified 5-fold-cross-validation
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [34]:
# Build the model 
from sklearn.svm import SVC

svm_model = SVC(random_state=42)

In [35]:
# Store metrics and predictions
all_y_true = []
all_y_pred = []
fold_metrics = []

In [36]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(kf.split(X_res, y_res)):

    X_train, X_val = X_res[train_idx], X_res[val_idx]
    y_train, y_val = y_res[train_idx], y_res[val_idx]

    # Fit and predict
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_val)

    # Save predictions for final report
    all_y_true.extend(y_val)
    all_y_pred.extend(y_pred)

    # Calculate metrics for the fold
    acc = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred)
    rec = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    fold_metrics.append({
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1': f1
    })

In [37]:
# Create a DataFrame with fold results
metrics_df = pd.DataFrame(fold_metrics)

In [38]:
# Print fold-by-fold results
print("Performance Metrics across each fold:")
print(metrics_df)

Performance Metrics across each fold:
   Accuracy  Precision    Recall        F1
0  0.772932   0.877339  0.634586  0.736475
1  0.774436   0.889126  0.627068  0.735450
2  0.750376   0.864333  0.593985  0.704100
3  0.754703   0.893023  0.578313  0.702011
4  0.756208   0.914842  0.565414  0.698885


In [39]:
# Print average metrics
print("Mean of Metrics across 5 folds:")
print(metrics_df.mean(numeric_only=True))

Mean of Metrics across 5 folds:
Accuracy     0.761731
Precision    0.887732
Recall       0.599873
F1           0.715384
dtype: float64


In [40]:
# Generate a final classification report using all predictions
print("Final Classification Report (Aggregated across all folds):")
print(classification_report(all_y_true, all_y_pred, digits=4))

Final Classification Report (Aggregated across all folds):
              precision    recall  f1-score   support

           0     0.6977    0.9236    0.7949      3324
           1     0.8870    0.5999    0.7157      3324

    accuracy                         0.7617      6648
   macro avg     0.7924    0.7617    0.7553      6648
weighted avg     0.7924    0.7617    0.7553      6648

